In [14]:
import redis

ip = 'localhost'

r0 = redis.Redis(host=ip, port=6379, db=0)
r1 = redis.Redis(host=ip, port=6379, db=1)


In [101]:
print json.loads(r0.get(1500))['batchTag']

[{u'tagTime': 1511774645.213363, u'scenario': {u'DIIS2': u'SEG001', u'DIIS3': u'01', u'DIIS1': u'S'}, u'isValid': u'Y', u'tagId': u'SEG001', u'securityFilter': u'admin', u'expireTime': 1511774675.213364, u'tagValue': 7, u'dataSource': u'EBM', u'tagName': [u'\u5ba2\u7fa41'], u'recommendWeight': u'1'}, {u'tagTime': 1511774645.215994, u'scenario': {u'DIIS2': u'SEG002', u'DIIS3': u'02', u'DIIS1': u'S'}, u'isValid': u'Y', u'tagId': u'SEG002', u'securityFilter': u'admin', u'expireTime': 1511774675.215994, u'tagValue': 3, u'dataSource': u'EBM', u'tagName': [u'\u5ba2\u7fa42'], u'recommendWeight': u'1'}]


In [5]:
def offer_tag_db_query(TAG_VALUE, sql_db):
    SQL = "SELECT tag_info FROM offer_tag_db WHERE tag_info->>'tag_value' = '{}';".format(TAG_VALUE)
    sql_db.execute(SQL)
    records = sql_db.fetchall()[0][0]
    return records

In [3]:
import psycopg2
conn = psycopg2.connect(database = "MJ",
                        host = "localhost",
                        user = "llc",
                        password = "")
cur = conn.cursor()

In [80]:
value = 1
offer_list = {"tag_value":value, "offer_list":offer_tag_db_query(1, cur)['offer_list']}
offer_list

{'offer_list': [u'OFF0012',
  u'OFF0057',
  u'OFF0055',
  u'OFF0032',
  u'OFF0047',
  u'OFF0019',
  u'OFF0008',
  u'OFF0007',
  u'OFF0031',
  u'OFF0059'],
 'tag_value': 1}

In [91]:
import json
import re
tmp = r1.get(1)
tmp = tmp.replace("'",'"')
test = re.sub(ur'u"','"',tmp)
json.loads(test)

{u'offer_list': [u'OFF0012',
  u'OFF0057',
  u'OFF0055',
  u'OFF0032',
  u'OFF0047',
  u'OFF0019',
  u'OFF0008',
  u'OFF0007',
  u'OFF0031',
  u'OFF0059'],
 u'tag_value': 1}

In [90]:
b = {"tag_value":1, "offer_list":r1.get(1)}
b=b.replace("'",'"')
b=re.sub(ur'u"','"',b)
json.loads(b)

AttributeError: 'dict' object has no attribute 'replace'

In [58]:
a = [u'OFF0031', u'OFF0058', u'OFF0062', u'OFF0056', u'OFF0008', u'OFF0061', u'OFF0021', u'OFF0011', u'OFF0007', u'OFF0060']

In [61]:
re.sub(ur"u'",'"',a)

TypeError: expected string or buffer

In [108]:
import psycopg2


# connect PostgreSQL
conn = psycopg2.connect(database = "MJ",
                        host = "localhost",
                        user = "llc",
                        password = "")
cur = conn.cursor()

In [109]:
def offer_label_db_query(offerid, sql_db):
    SQL = "SELECT tag_info FROM offer_label_db WHERE tag_info->>'offer_id' = '{}';".format(offerid)
    sql_db.execute(SQL)
    records = sql_db.fetchall()[0][0]
    return records

In [110]:
offer_label_db_query("OFF0001",cur)

{u'label': [u'\u5916\u5e63',
  u'\u7db2\u8def\u63db\u532f',
  u'\u7db2\u8def\u4ea4\u6613',
  u'\u532f\u7387',
  u'\u512a\u60e0',
  u'\u7db2\u8def\u9280\u884c',
  u'\u884c\u52d5\u9280\u884c',
  u'\u591c\u9593\u63db\u532f'],
 u'offer_id': u'OFF0001'}

In [112]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
from datetime import datetime
import time
import json
import psycopg2
import redis
import sys

reload(sys)
sys.setdefaultencoding('utf8') 

intent_table = pd.read_excel("INTENT_TAG.xlsx")
intent_tag = list(intent_table)

intent_table['intent_list']=''
intent_Json = []
for i in range(intent_table.shape[0]):
    temp_detail = {}
    for column in intent_tag:
        temp_detail[column]=str(intent_table.loc[i,:][column])
    
    scenario = intent_table.scenario[i].split('|')

    temp_detail['scenario'] = {'DIIS1' : '',
                               'DIIS2' : '',
                               'DIIS3' : ''}
    
    for j in range(len(scenario)):
        DIIS = 'DIIS' + str(j+1)
        temp_detail['scenario'][DIIS] = scenario[j]
    
    intent_Json.append(temp_detail)

    
    
database='test'
host = 'localhost'
user = 'llc'
password = ''

conn = psycopg2.connect(database = database,
                        host = host,
                        user = user,
                        password = password)
cur = conn.cursor()

create_intent_db_sql = """CREATE TABLE intent_db( 
            id serial PRIMARY KEY,   
            tag_info json NOT NULL   
            )"""
cur.execute(create_intent_db_sql)
conn.commit()

for i in intent_Json:
    SQL = "INSERT INTO test_db(tag_info) VALUES('{}' );".format(json.dumps(i))
    cur.execute(SQL)
    conn.commit()

In [129]:
conn = psycopg2.connect(database = "MJ",
                        host = 'localhost',
                        user = "llc",
                        password = "")
cur = conn.cursor() 

cur.execute("select * from information_schema.tables where table_name='intent_db';")
if bool(cur.rowcount):
    print 'intent_db is exist!'
    pass

else:
    SQL = "CREATE TABLE intent_db(id serial PRIMARY KEY,tag_info json NOT NULL);"
    cur.execute(SQL)
    conn.commit()
    print "Create Table Done!"